# ML Dataset

First attempt at an ML dataset.
Here one example will be one forecast date, and the corresponding observations.

We will use Arlan's top ten as features: https://docs.google.com/spreadsheets/d/1CzNvLExxJYhFAS_bWs97DON2xpG3zmDt-HHwQhqjpkE/edit#gid=0.

- *Surface Air Temperature (at 2 meter)*
- *Total precipitation*
- Soil moisture top 20 cm
- Snow Depth Water equivalent
- Sea Surface Temperature
- Sea Ice Cover
- Mean Sea Level Pressure
- Geopotential height@ 1000 hPa
- Geopotential height@ 200 hPa
- U-velocity (aka Zonal wind )@ 200 hPa
- U-velocity (aka Zonal wind )@ 850 hPa
- V-velocity (Meridional wind)@ 200 hPa
- V-velocity (Meridional wind)@ 850 hPa





In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask.distributed
import logging
import pathlib
import xarray as xr

from crims2s.data import normalize_dataset
from crims2s.dask import create_dask_cluster
from crims2s.util import fix_dataset_dims

In [ ]:
_logger = logging.getLogger(__name__)

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
_logger

## Initialize dask cluster

In [ ]:
cluster = create_dask_cluster()

In [ ]:
cluster.scale(jobs=2)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
INPUT_DATA = '***BASEDIR***/training-input/'
INPUT_DATA_SPLIT_PLEV = '***BASEDIR***processed/training-input'
OBS_DATA = '***BASEDIR***/processed/training-output-reference/'
OBS_FILE_TEST = '***BASEDIR***/renku/forecast-like-observations_2020_biweekly_terciled.nc'
TRAIN_OBS = '***BASEDIR***/renku/hindcast-like-observations_2000-2019_biweekly_terciled.nc'
OUTPUT_DIR = '***BASEDIR***/training/2021-07-24-first'


CENTER = 'ecmwf'

In [ ]:
output_path = pathlib.Path(OUTPUT_DIR)
output_path.mkdir(exist_ok=True, parents=True)

In [ ]:
FLAT_FIELDS = ['t2m', 'tp', 'sm20', 'sst', 'ci', 'msl']
MULTILEVEL_FIELDS = [('u', [200, 850]), ('v', [200, 850]), ('gh', [1000, 200])]

In [ ]:
d = xr.open_dataset('***BASEDIR***processed/training-input/ecmwf-hindcast-u200-20200528.nc')

In [ ]:
d.plev[0]

In [ ]:
new_d = d
for var in d.data_vars:
    level = int(d.plev[0])
    new_d = new_d.rename({var: f'{var}{level}})

In [ ]:
level = int(d.plev[0])
new_names = {k: f'{k}{level}' for k in d.data_vars}
new_names

In [ ]:
d.rename(new_names)

In [ ]:
train_obs_terciled.to_array().isel(forecast_time=0, category=0, lead_time=0)

In [ ]:
def preprocess_single_level_file(d):
    level = int(d.plev[0])
    new_names = {k: f'{k}{level}' for k in d.data_vars}
    
    return d.rename(new_names).isel(plev=0).drop('plev')

def process_one_forecast_week(datestring, obs_terciled):
    filenames = [f'{INPUT_DATA}/{CENTER}-hindcast-{f}-{forecast_times[0]}.nc' for f in flat_fields]
    
    # Flat fields.
    _logger.debug('Opening flat fields')
    flat_dataset = xr.open_mfdataset(filenames, preprocess=fix_dataset_dims).isel(depth_below_and_layer=0, meanSea=0).drop(['depth_below_and_layer', 'meanSea'])
    
    # Non-flat fields.
    _logger.debug('Opening larger fields')
    flattened_fields = []
    for field, levels in multilevel_fields:
        files = [f'{INPUT_DATA_SPLIT_PLEV}/{CENTER}-hindcast-{field}{level}-{forecast_times[0]}.nc' for level in levels]
        _logger.debug(f'Opening multi-level fields: {files}')
        ds = xr.open_mfdataset(files, preprocess=preprocess_single_level_file)
        flattened_fields.append(ds)
            
    # Merge the two.
    ds = normalize_dataset(xr.merge([*flattened_fields, flat_dataset]))
    ds = ds.persist()
    
    # Export all years separately.
    examples = []
    for i in range(ds.dims['forecast_year']):
        print(i)
        to_export_x = ds.isel(forecast_year=i, forecast_dayofyear=0).to_array().rename('x').transpose('lead_time', 'latitude', 'longitude', 'realization', 'variable')
        to_export_y = obs_terciled.sel(forecast_time=to_export_x.forecast_time).to_array().rename('y').transpose('latitude', 'longitude', 'variable', 'lead_time', 'category')
        examples.append((to_export_x, to_export_y))
    
    return examples

In [ ]:
train_obs_terciled = xr.open_dataset(TRAIN_OBS)

In [ ]:
input_path = pathlib.Path(INPUT_DATA)
forecast_times = [x.stem.split('-')[-1] for x in input_path.iterdir() if 't2m' in x.stem and CENTER in x.stem]

In [ ]:
examples = process_one_forecast_week(forecast_times[0], train_obs_terciled)

In [ ]:
examples[10][1]

In [ ]:
for x, y in examples:
    forecast_time = y.forecast_time
    year = int(forecast_time.dt.year)
    month = int(forecast_time.dt.month)
    day = int(forecast_time.dt.day)
    filename = f'train_example_{year:04}{month:02}{day:02}.nc'
    
    output_file = output_path / filename
    
    x.to_netcdf(output_file, group='/x', mode='w')
    y.to_netcdf(output_file, group='/y', mode='a')

In [ ]:
y.isel(variable=0, lead_time=0, category=0).plot()

In [ ]:
!ls -lh {OUTPUT_DIR}

In [ ]:
tstt = xr.open_dataset(output_file, group='/y')

In [ ]:
tstt

In [ ]:
y

In [ ]:
forecast_times[:5]

In [ ]:
filenames = [f'{INPUT_DATA}/{CENTER}-hindcast-{f}-{forecast_times[0]}.nc' for f in flat_fields]
flat_dataset = xr.open_mfdataset(filenames, preprocess=fix_dataset_dims).isel(depth_below_and_layer=0, meanSea=0).drop(['depth_below_and_layer', 'meanSea'])

In [ ]:
flattened_fields = []

for field, levels in multilevel_fields:
    filename = f'{INPUT_DATA}/{CENTER}-hindcast-{field}-{forecast_times[0]}.nc'
    ds = fix_dataset_dims(xr.open_dataset(filename))
    print(ds)
    for level in levels:
        flattened_fields.append(ds.sel(plev=level).drop('plev').rename({field: f'{field}{level}'}))

In [ ]:
flattened_dataset = xr.merge(flattened_fields)

In [ ]:
ds = normalize_dataset(xr.merge([flattened_dataset, flat_dataset]))

In [ ]:
ds

In [ ]:
ds.mean().compute()

In [ ]:
ds.std().compute()

## Make examples from bigger dataset

In [ ]:
to_export_x = ds.isel(forecast_year=0).to_array().isel(forecast_dayofyear=0).transpose('lead_time', 'latitude', 'longitude', 'realization', 'variable')

In [ ]:
to_export_x

In [ ]:
to_export_x.to_netcdf('***BASEDIR***/mldataset/')

## Already terciled observations

In [ ]:
TRAIN_OBS = '***BASEDIR***/renku/hindcast-like-observations_2000-2019_biweekly_terciled.nc'
train_obs_terciled = xr.open_dataset(TRAIN_OBS)

In [ ]:
train_obs_terciled